In [42]:
import json
from pathlib import Path

import pandas as pd
import numpy as np

### Exp 1-3: varying number on gaussians on full model, no 4DSH, and no 4D rotation

In [101]:
base_path = Path('/data-kp/jgajardo/results/4d-gaussian-splatting')
experiment_paths = [
    base_path / 'output/WAT/realtime4DGS-exp1/force_sh_3d_false',
    base_path / 'output/WAT/realtime4DGS-exp1/force_sh_3d_true',
    base_path / 'output/WAT/realtime4DGS-exp2/gaussians_500000',
    base_path / 'output/WAT/realtime4DGS-exp2/gaussians_1000000',
    base_path / 'output/WAT/realtime4DGS-exp2/gaussians_3000000',
    base_path / 'output/WAT/realtime4DGS-exp2/gaussians_2000000',
    base_path / 'output/WAT/realtime4DGS-exp2/gaussians_4000000',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_500000/force_sh_3d_true',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_500000/force_sh_3d_true',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_500000/rot_4d_false',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_1000000/force_sh_3d_true',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_1000000/rot_4d_false',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_2000000/force_sh_3d_true',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_2000000/rot_4d_false',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_3000000/force_sh_3d_true',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_3000000/rot_4d_false',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_4000000/force_sh_3d_true',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_4000000/rot_4d_false',
    base_path / 'output/WAT/realtime4DGS-exp3/gaussians_unlimited/rot_4d_false',
]

In [102]:
def get_results_per_scene(results_path: Path):
    results = []
    for scene in sorted(results_path.iterdir()):
        # Storage
        checkpoint = scene / 'chkpnt30000.pth'
        if not checkpoint.exists():
            print(f"{checkpoint.name} for scene {scene.name} doesn't exist")
            continue
        storage_gb= round(checkpoint.stat().st_size / 1024**3, 3)

        # Wanbd metrics
        wandb_run_path = scene / 'wandb' / 'latest-run' / 'files'
        metadata_json_path = wandb_run_path / 'wandb-metadata.json'
        summary_json_path = wandb_run_path / 'wandb-summary.json'

        if not metadata_json_path.exists() or not summary_json_path.exists():
            print(f"Wandb json paths can't be found for {scene.name}")

        with open(summary_json_path, 'r') as f:
            data = json.load(f)
            runtime = int(data.get('_runtime') / 60)
            total_gaussians = data.get('total_points', 'N/A')

        # NVS metrics
        json_path = scene / 'results.json'
        if not json_path.exists():
            print(f"{json_path.name} for scene {json_path.parent.name} doesn't exist")
        with open(json_path, 'r') as f:
            data = json.load(f)
            data = data.get('ours_None', {})
            psnr = data.get('PSNR', 'N/A')
            ssim = data.get('SSIM', 'N/A')
            lpips = data.get('LPIPS', 'N/A')
            
        metrics = {
            'Scene': scene.name,
            'PSNR': psnr,
            'SSIM': ssim,
            'LPIPS': lpips,
            'Runtime (min)': runtime,
            'Total Gaussians': total_gaussians,
            'Storage (GB)': storage_gb,
            }
        
        results.append(metrics)

    numeric_keys = ['PSNR', 'SSIM', 'LPIPS', 'Runtime (min)', 'Total Gaussians', 'Storage (GB)']
    avg_metrics = {'Scene': 'Average'}
    std_metrics = {'Scene': 'Std Dev'}

    for key in numeric_keys:
        # Convert values to float, excluding 'N/A' values
        values = [float(r[key]) for r in results if r[key] != 'N/A']
        
        if values:  # Only calculate if we have valid values
            avg_metrics[key] = round(np.mean(values), 3)
            std_metrics[key] = round(np.std(values), 3)
        else:
            avg_metrics[key] = 'N/A'
            std_metrics[key] = 'N/A'

    # Append summary rows to results
    results.append(avg_metrics)
    results.append(std_metrics)

    return pd.DataFrame(results)

In [ ]:
output_base_path = Path('../results/2_ablations_gaussians_and_3DSH-3DRot')
for results_path in experiment_paths:
    results = get_results_per_scene(results_path)
    csv_filename = '_'.join(str(results_path.relative_to(base_path)).split('/')[1:])
    if 'exp1' in csv_filename:
        parts = csv_filename.split('_')
        parts.insert(2, 'gaussians_unlimited')
        csv_filename = '_'.join(parts)
    elif 'exp2' in csv_filename:
        csv_filename += '_force_sh_3d_false'
    print(f'Writting results file for {csv_filename}.csv')
    results.to_csv(f'{csv_filename}.csv', index=False)